In [1]:
from rosemary import jpt_parse_args, jpt_setup; jpt_setup()

import sys
sys.path.append('/dccstor/mit_fm/wpq/github/mitibm2023/external/open-instruct/')

/dccstor/mit_fm/wpq/github/rosemary/src/rosemary/__init__.py:25: UserWarning: Install `torch` for functionalities dependent on torch
  warn(f'Install `torch` for functionalities dependent on torch')


In [6]:
import logging
import os
import sys
from dataclasses import dataclass, field
from typing import Optional
from functools import partial
import datasets
import torch
from datasets import load_dataset

import transformers
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    LlamaTokenizer,
    LlamaTokenizerFast,
    HfArgumentParser,
    TrainingArguments,
    DataCollatorForSeq2Seq,
    set_seed,
    GPTNeoXTokenizerFast,
    GPT2Tokenizer,
    GPT2TokenizerFast, 
    OPTForCausalLM,
)
from peft import LoraConfig, TaskType, get_peft_model
from transformers.trainer_utils import get_last_checkpoint


In [62]:
from open_instruct.finetune_trainer import ModelArguments, DataTrainingArguments
from open_instruct.finetune_trainer import encode_with_prompt_completion_format, encode_with_messages_format
from open_instruct.finetune_trainer import SafeSaveTrainer

In [20]:
logger = logging.getLogger(__name__)

In [37]:

cmd = """
    --model_name_or_path huggyllama/llama-7b \
    --tokenizer_name huggyllama/llama-7b \
    --use_fast_tokenizer True \
    --train_file data/processed/flanv2_cot_oasst1_dolly.jsonl \
    --max_seq_length 2048 \
    --use_lora \
    --lora_rank 8 \
    --lora_alpha 8 \
    --lora_dropout 0.05 \
    --do_train \
    --preprocessing_num_workers 16 \
    --per_device_train_batch_size 1 \
    --gradient_accumulation_steps 128 \
    --gradient_checkpointing \
    --learning_rate 2e-5 \
    --lr_scheduler_type linear \
    --warmup_ratio 0.03 \
    --weight_decay 0. \
    --evaluation_strategy "no" \
    --logging_steps 1 \
    --save_strategy steps \
    --save_steps 100 \
    --save_total_limit 1 \
    --num_train_epochs 2 \
    --bf16 True \
    --tf32 True \
    --report_to tensorboard \
    --torch_dtype bfloat16 \
    --dataloader_num_workers 8 \
    --output_dir "results/huggyllama:llama-7b+lora(r=8,a=8)_humanmix"
"""
#     --overwrite_output_dir \
import shlex
args = shlex.split(cmd)


parser = HfArgumentParser((ModelArguments, DataTrainingArguments, TrainingArguments))
model_args, data_args, training_args = parser.parse_args_into_dataclasses(args)
model_args, data_args, training_args

(ModelArguments(model_name_or_path='huggyllama/llama-7b', config_name=None, tokenizer_name='huggyllama/llama-7b', cache_dir=None, use_fast_tokenizer=True, model_revision='main', use_auth_token=False, torch_dtype='bfloat16', use_lora=True, lora_rank=8, lora_alpha=8.0, lora_dropout=0.05, load_in_8bit=False),
 DataTrainingArguments(dataset_name=None, dataset_config_name=None, train_file='data/processed/flanv2_cot_oasst1_dolly.jsonl', max_train_samples=None, streaming=False, overwrite_cache=False, preprocessing_num_workers=16, max_seq_length=2048),
 TrainingArguments(
 _n_gpu=1,
 adafactor=False,
 adam_beta1=0.9,
 adam_beta2=0.999,
 adam_epsilon=1e-08,
 auto_find_batch_size=False,
 bf16=True,
 bf16_full_eval=False,
 data_seed=None,
 dataloader_drop_last=False,
 dataloader_num_workers=8,
 dataloader_pin_memory=True,
 ddp_backend=None,
 ddp_bucket_cap_mb=None,
 ddp_find_unused_parameters=None,
 ddp_timeout=1800,
 debug=[],
 deepspeed=None,
 disable_tqdm=False,
 do_eval=False,
 do_predict=Fal

In [38]:

# Detecting last checkpoint.
last_checkpoint = None
if os.path.isdir(training_args.output_dir) and training_args.do_train and not training_args.overwrite_output_dir:
    last_checkpoint = get_last_checkpoint(training_args.output_dir)
    if last_checkpoint is None and len(os.listdir(training_args.output_dir)) > 0:
        raise ValueError(
            f"Output directory ({training_args.output_dir}) already exists and is not empty. "
            "Use --overwrite_output_dir to overcome."
        )
    elif last_checkpoint is not None and training_args.resume_from_checkpoint is None:
        logger.info(
            f"Checkpoint detected, resuming training at {last_checkpoint}. To avoid this behavior, change "
            "the `--output_dir` or add `--overwrite_output_dir` to train from scratch."
        )
        
last_checkpoint

'results/huggyllama:llama-7b+lora(r=8,a=8)_humanmix/checkpoint-1800'

In [40]:

# Set seed before initializing model.
set_seed(training_args.seed)

if data_args.dataset_name is not None:
    # Downloading and loading a dataset from the hub.
    raw_datasets = load_dataset(
        data_args.dataset_name,
        data_args.dataset_config_name,
        cache_dir=model_args.cache_dir,
        use_auth_token=True if model_args.use_auth_token else None,
        streaming=data_args.streaming,
    )
else:
    data_files = {}
    dataset_args = {}
    if data_args.train_file is not None:
        data_files["train"] = data_args.train_file
    raw_datasets = load_dataset(
        "json",
        data_files=data_files,
        cache_dir=model_args.cache_dir,
        use_auth_token=True if model_args.use_auth_token else None,
        **dataset_args,
    )

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /dccstor/mit_fm/wpq/hf_cache/datasets/json/default-712ef65df98800fb/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [51]:

config_kwargs = {
    "cache_dir": model_args.cache_dir,
    "revision": model_args.model_revision,
    "use_auth_token": True if model_args.use_auth_token else None,
}
if model_args.config_name:
    config = AutoConfig.from_pretrained(model_args.config_name, **config_kwargs)
elif model_args.model_name_or_path:
    config = AutoConfig.from_pretrained(model_args.model_name_or_path, **config_kwargs)
    # wpq: add support for mpt models.
    if 'mpt' in model_args.model_name_or_path:
        config.attn_config['attn_impl'] = 'triton'
        config.init_device = 'cuda' # For fast initialization directly on GPU!
else:
    raise ValueError(
        "You are instantiating a new config instance from scratch. This is not supported by this finetuning script."
    )

tokenizer_kwargs = {
    "cache_dir": model_args.cache_dir,
    "use_fast": model_args.use_fast_tokenizer,
    "revision": model_args.model_revision,
    "use_auth_token": True if model_args.use_auth_token else None,
}
if model_args.tokenizer_name:
    tokenizer = AutoTokenizer.from_pretrained(model_args.tokenizer_name, **tokenizer_kwargs)
elif model_args.model_name_or_path:
    tokenizer = AutoTokenizer.from_pretrained(model_args.model_name_or_path, **tokenizer_kwargs)
else:
    raise ValueError(
        "You are instantiating a new tokenizer from scratch. This is not supported by this finetuning script."
    )


In [52]:

if model_args.model_name_or_path:
    torch_dtype = (
        model_args.torch_dtype
        if model_args.torch_dtype in ["auto", None]
        else getattr(torch, model_args.torch_dtype)
    )
    model = AutoModelForCausalLM.from_pretrained(
        model_args.model_name_or_path,
        from_tf=bool(".ckpt" in model_args.model_name_or_path),
        config=config,
        cache_dir=model_args.cache_dir,
        revision=model_args.model_revision,
        use_auth_token=True if model_args.use_auth_token else None,
        torch_dtype=torch_dtype,
        # wpq: 8bit training
        load_in_8bit=model_args.load_in_8bit,
        trust_remote_code=bool('mpt' in model_args.model_name_or_path),
    )
else:
    logger.warning("No pretrained model_name_or_path is given. Training new model from scratch.")
    model = AutoModelForCausalLM.from_config(config)
    n_params = sum({p.data_ptr(): p.numel() for p in model.parameters()}.values())
    logger.info(f"Training new model from scratch - Total size={n_params/2**20:.2f}M params")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [54]:

# no default pad token for llama!
# here we add all special tokens again, because the default ones are not in the special_tokens_map 
if isinstance(tokenizer, (LlamaTokenizer, LlamaTokenizerFast)):
    num_added_tokens = tokenizer.add_special_tokens({
        "bos_token": "<s>",
        "eos_token": "</s>",
        "unk_token": "<unk>",
        "pad_token": "<pad>",
    })
    assert num_added_tokens in [0, 1], "LlamaTokenizer/ LlamaTokenizerFast should only add one special token - the pad_token, or no tokens if pad token present."
elif isinstance(tokenizer, GPTNeoXTokenizerFast):
    num_added_tokens = tokenizer.add_special_tokens({
        "pad_token": "<pad>",
    })
    assert num_added_tokens == 1, "GPTNeoXTokenizer should only add one special token - the pad_token."
elif isinstance(tokenizer, (GPT2Tokenizer, GPT2TokenizerFast)) and isinstance(model, OPTForCausalLM):
    num_added_tokens = tokenizer.add_special_tokens({'unk_token': '<unk>'})
## wpq: add support for gpt2 tokenizer.
elif isinstance(tokenizer, (GPT2Tokenizer, GPT2TokenizerFast)):
    num_added_tokens = tokenizer.add_special_tokens({
        "pad_token": "<pad>",
    })
    assert num_added_tokens == 1, "GPT2Tokenizer should only add one special token - the pad_token."

# resize embeddings if needed (e.g. for LlamaTokenizer)
embedding_size = model.get_input_embeddings().weight.shape[0]
if len(tokenizer) > embedding_size:
    model.resize_token_embeddings(len(tokenizer))

    
# wpq: use int8 training
# wpq: put this after resize embedding!
if model_args.load_in_8bit:
    from peft import prepare_model_for_int8_training
    model = prepare_model_for_int8_training(model)

In [56]:

# wpq: add peft to finetune_trainer.py
if model_args.use_lora:
    logger.info("Initializing LORA model...")
    peft_config = LoraConfig(
        task_type=TaskType.CAUSAL_LM, 
        inference_mode=False, 
        bias='none',
        r=model_args.lora_rank, 
        lora_alpha=model_args.lora_alpha, 
        lora_dropout=model_args.lora_dropout,
        target_modules=['q_proj','k_proj','v_proj','o_proj'],
    )
    # wpq: the following fixes `element 0 of tensors does not require grad and does not have a grad_fn` 
    # https://github.com/huggingface/peft/issues/137
    # https://github.com/huggingface/peft/issues/522
    if hasattr(training_args, 'gradient_checkpointing'):
        if training_args.gradient_checkpointing:
            model.enable_input_require_grads()

    model = get_peft_model(model, peft_config)
    model.print_trainable_parameters()

trainable params: 8388608 || all params: 6746812416 || trainable%: 0.12433438908285782


In [ ]:

# Preprocessing the datasets.
if "prompt" in raw_datasets["train"].column_names and "completion" in raw_datasets["train"].column_names:
    encode_function = partial(
        encode_with_prompt_completion_format,
        tokenizer=tokenizer,
        max_seq_length=data_args.max_seq_length,
    )
elif "messages" in raw_datasets["train"].column_names:
    encode_function = partial(
        encode_with_messages_format,
        tokenizer=tokenizer,
        max_seq_length=data_args.max_seq_length,
    )
else:
    raise ValueError("You need to have either 'prompt'&'completion' or 'messages' in your column names.")

    

# To speed up this part, we use multiprocessing.
with training_args.main_process_first(desc="Processing instruction data"):
    if not data_args.streaming:
        lm_datasets = raw_datasets.map(
            encode_function,
            batched=False,
            num_proc=data_args.preprocessing_num_workers,
            load_from_cache_file=not data_args.overwrite_cache,
            desc="Tokenizing and reformatting instruction data",
        )
    else:
        lm_datasets = raw_datasets.map(
            encode_function,
            batched=False,
        )
    lm_datasets.set_format(type="pt")

if training_args.do_train:
    if "train" not in raw_datasets:
        raise ValueError("--do_train requires a train dataset")
    train_dataset = lm_datasets["train"]
    if data_args.max_train_samples is not None:
        max_train_samples = min(len(train_dataset), data_args.max_train_samples)
        train_dataset = train_dataset.select(range(max_train_samples))


Tokenizing and reformatting instruction data (num_proc=16):   0%|          | 0/270679 [00:00<?, ? examples/s]

In [74]:
from transformers.trainer import (
    CONFIG_NAME, 
    ADAPTER_WEIGHTS_NAME, 
    ADAPTER_SAFE_WEIGHTS_NAME, 
    WEIGHTS_INDEX_NAME,
    SAFE_WEIGHTS_NAME,
    SAFE_WEIGHTS_INDEX_NAME)
from transformers import PretrainedConfig
from transformers import __version__
from transformers.utils import is_safetensors_available, is_peft_available
if is_safetensors_available():
    import safetensors.torch
if is_peft_available():
    from peft import PeftModel
from transformers.modeling_utils import load_sharded_checkpoint



In [75]:
from open_instruct.finetune_trainer import SafeSaveTrainer

# initalize a trainer
# here we use a custom trainer that moves the model to CPU when saving the checkpoint in FSDP mode
# we can switch to the default trainer after moving to deepspeed (let's don't change too much for now)
trainer = SafeSaveTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset if training_args.do_train else None,
    tokenizer=tokenizer,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model),
)

In [76]:

checkpoint = None
if training_args.resume_from_checkpoint is not None:
    checkpoint = training_args.resume_from_checkpoint
elif last_checkpoint is not None:
    checkpoint = last_checkpoint
train_result = trainer.train(resume_from_checkpoint=checkpoint)

/dccstor/mit_fm/miniconda/envs/open-instruct/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to

Step,Training Loss



KeyboardInterrupt



In [ ]:

trainer.save_model()  # Saves the tokenizer too for easy upload

metrics = train_result.metrics

max_train_samples = (
    data_args.max_train_samples if data_args.max_train_samples is not None else len(train_dataset)
)
metrics["train_samples"] = min(max_train_samples, len(train_dataset))

trainer.log_metrics("train", metrics)
trainer.save_metrics("train", metrics)
trainer.save_state()